In [2]:
import sys
sys.path.append('/home/rhino/anaconda3/envs/rl_env/lib/python3.9/site-packages')
# 구현에 사용할 패키지 임포트
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import gym

In [43]:
from collections import namedtuple


# example using namedtuple
Tr = namedtuple('tr', ('name_a', 'value_b'))
Tr_object = Tr('nameA', 100)

print(Tr_object)
print(Tr_object.value_b)




# Execute DQN
import random
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

tt = namedtuple('tt', ('A', 'B', 'C', 'D'))
tt_tmp = []
for i in range(100):
    tt_tmp.append((torch.FloatTensor([[i*1, i*1.5, i*2.0, i*2.5]]), torch.FloatTensor([[i*2]]), \
                   torch.FloatTensor([[i*3]]), torch.LongTensor([[i*4]])))
    
    
print("-----------------------------------------")
# tt_sample = random.sample(tt_tmp, 4)
# print(tt_sample)

# print("zip=", *zip(*tt_sample))

# print("tt=", tt(*zip(*tt_sample)))

# tt_sample = tt(*zip(*tt_sample))


# print()

# print("tt_sample.A = ", tt_sample.A)
# print("cat = ", torch.cat(tt_sample.A))

# A = torch.cat(tt_sample.A)
# B = torch.cat(tt_sample.B)
# C = torch.cat(tt_sample.C)
# D = torch.cat(tt_sample.D)

print("-----------------------------------------")
        
# model = nn.Sequential()
# model.add_module('fc1', nn.Linear(4, 32))
# model.add_module('relu1', nn.ReLU())
# model.add_module('fc2', nn.Linear(32, 2))

# print(index)

# print("model = ", model)
# print("A = ", A)
# print("D = ", D)

# model.eval()
# print(model(A))

# matrix = torch.range(0,99).reshape(10,10)
# indices = [0,1,2,3,4,5,6,7,8,9]
# indices = torch.tensor(indices).unsqueeze(axis=1)
# print(matrix)
# print(indices)
# print(matrix.shape)
# print(torch.gather(matrix, 0, indices ))
# x = torch.tensor(2.0, requires_grad=True)
# y = 9*x**4 + 2*x**3 + 3*x**2 + 6*x + 1
# y.backward()
# print(x.grad)

# x = torch.randn(2, 2, requires_grad=True)
# print(x)
# y = x + 2
# print(y)
# z = (y * y)
# print(z)
# y.backward(z)

# print(x.grad)
# # tensor([[2.2796, 3.2123],
# #         [5.1224, 0.6321]])

# v = torch.arange(0, 5)
# print("v.shape : ", v.shape)
# print(v.size())

A = torch.arange(25).reshape(5, 5)
# tensor([[ 0,  1,  2,  3,  4],
#         [ 5,  6,  7,  8,  9],
#         [10, 11, 12, 13, 14],
#         [15, 16, 17, 18, 19],
#         [20, 21, 22, 23, 24]])
indices = torch.tensor([
    [0, 1, 2],
    [1, 2, 3],
    [2, 3, 3],
    [3, 4, 1],
    [0, 0, 0]
])

print(torch.gather(A, 0, indices))

A = torch.arange(27).reshape(3, 3, 3)
# tensor([[[ 0,  1,  2],
#          [ 3,  4,  5],
#          [ 6,  7,  8]],

#         [[ 9, 10, 11],
#          [12, 13, 14],
#          [15, 16, 17]],

#         [[18, 19, 20],
#          [21, 22, 23],
#          [24, 25, 26]]])

indices = torch.tensor([
    [[0,0,0],[1,1,1],[2,2,2]],
    [[1,1,1],[2,2,2,],[0,0,0]],
    [[0,1,2],[0,1,2],[0,1,2]]])
    
print(torch.gather(A, 2, indices))


tr(name_a='nameA', value_b=100)
100
-----------------------------------------
-----------------------------------------
tensor([[ 0,  6, 12],
        [ 5, 11, 17],
        [10, 16, 17],
        [15, 21,  7],
        [ 0,  1,  2]])
tensor([[[ 0,  0,  0],
         [ 4,  4,  4],
         [ 8,  8,  8]],

        [[10, 10, 10],
         [14, 14, 14],
         [15, 15, 15]],

        [[18, 19, 20],
         [21, 22, 23],
         [24, 25, 26]]])


In [23]:
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

In [70]:
# 상수 정의
ENV = 'CartPole-v0'  # 태스크 이름
GAMMA = 0.99  # 시간할인율
MAX_STEPS = 200  # 1에피소드 당 최대 단계 수
NUM_EPISODES = 500  # 최대 에피소드 수

In [71]:
# memory class for saving transition
class ReplayMemory:
    def __init__(self, CAPACITY):
        self.capacity = CAPACITY # Max size of memory
        self.memory = [] # parameter for stacking transition
        self.index = 0

    # 메모리에 Transition tuple에 cartpole의 정보를 저장한다.
    def push(self, state, action, state_next, reward):
        # save transition in memory
        if len(self.memory) < self.capacity:
            self.memory.append(None) # memory is not full
            
        self.memory[self.index] = Transition(state, action, state_next, reward)
        
        self.index = (self.index + 1) % self.capacity

    # Transition이 저장된 memory에서 batch_size만큼 랜덤으롤 샘플을 뽑는다.        
    def sample(self, batch_size):
        # Randomize by the number of batches
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        # return the number of transition
        return len(self.memory)

In [110]:
# Execute DQN
import random
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

BATCH_SIZE = 32
CAPACITY = 10000

class Brain:
    def __init__(self, num_states, num_actions):
        self.num_actions = num_actions # the number of action
        
        # make memory object to memorize transition
        self.memory = ReplayMemory(CAPACITY)
        
        # Neural Network Construction
        self.model = nn.Sequential()
        self.model.add_module('fc1', nn.Linear(num_states, 32))
        self.model.add_module('relu1', nn.ReLU())
        self.model.add_module('fc2', nn.Linear(32, 32))
        self.model.add_module('relu2', nn.ReLU())
        self.model.add_module('fc3', nn.Linear(32, num_actions))
        
        print(self.model) 
        
        # select optimization method
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001)
        
    def replay(self):
        # learnLearning joint weights of neural networks with Experience Replay
                
        # -----------------------------------------
        # 1. 저장된 transition 수 확인
        # -----------------------------------------
        # 1.1 저장된 transition의 수가 미니배치 크기보다 작으면 아무 것도 하지 않음
        if len(self.memory) < BATCH_SIZE:
            return
        
        # -----------------------------------------
        # 2. 미니배치 생성
        # -----------------------------------------
        # 2.1 메모리 객체에서 미니배치를 추출
        transitions = self.memory.sample(BATCH_SIZE)
        
        # 2.2 각 변수를 미니배치에 맞는 형태로 변형
        # transitions는 각 단계 별로 (state, action, state_next, reward) 형태로 BATCH_SIZE 갯수만큼 저장됨
        # 다시 말해, (state, action, state_next, reward) * BATCH_SIZE 형태가 된다
        # 이것을 미니배치로 만들기 위해
        # (state*BATCH_SIZE, action*BATCH_SIZE, state_next*BATCH_SIZE, reward*BATCH_SIZE) 형태로 변환한다
        batch = Transition(*zip(*transitions))
            
        # 2.3 각 변수의 요소를 미니배치에 맞게 변형하고, 신경망으로 다룰 수 있도록 Variable로 만든다
        # state를 예로 들면, [torch.FloatTensor of size 1*4] 형태의 요소가 BATCH_SIZE 갯수만큼 있는 형태이다
        # 이를 torch.FloatTensor of size BATCH_SIZE*4 형태로 변형한다
        # 상태, 행동, 보상, non_final 상태로 된 미니배치를 나타내는 Variable을 생성 
        # cat은 Concatenates(연접)을 의미한다       
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
        
        # -----------------------------------------
        # 3. 정답신호로 사용할 Q(s_t, a_t)를 계산
        # -----------------------------------------
        # 3.1 신경망을 추론 모드로 전환
        self.model.eval()
        
        # 3.2 신경망으로 Q(s_t, a_t)를 계산
        # self.model(state_batch)은 왼쪽, 오른쪽에 대한 Q값을 출력하며
        # [torch.FloatTensor of size BATCH_SIZEx2] 형태이다
        # 여기서부터는 실행한 행동 a_t에 대한 Q값을 계산하므로 action_batch에서 취한 행동 a_t가 
        # 왼쪽이냐 오른쪽이냐에 대한 인덱스를 구하고, 이에 대한 Q값을 gather 메서드로 모아온다        
        state_action_values = self.model(state_batch).gather(1, action_batch)
#         print("action_batch = ", action_batch)
#         print("self.model(state_batch) = ", self.model(state_batch))
        # 3.3 max{Q(s_t+1, a)}값을 계산한다 이때 다음 상태가 존재하는지에 주의해야 한다
        
        # cartpole이 done 상태가 아니고, next_state가 존재하는지 확인하는 인덱스 마스크를 만듬
        non_final_mask = torch.ByteTensor(tuple(map(lambda s: s is not None, batch.next_state)))
        
        # 먼저 전체를 0으로 초기화
        next_state_values = torch.zeros(BATCH_SIZE)
        
        # 다음 상태가 있는 인덱스에 대한 최대 Q값을 구한다
        # 출력에 접근하여 열방향 최대값(max(1))이 되는 [값, 인덱스]를 구한다
        # 그리고 이 Q값(인덱스=0)을 출력한 다음
        # detach 메서드로 이 값을 꺼내온다 
        next_state_values[non_final_mask] = self.model(non_final_next_states).max(1)[0].detach()
        
        # 3.4 정답신호로 사용할 Q(s_t, a_t)값을 Q러닝 식으로 계산한다
        expected_state_action_values = reward_batch + GAMMA * next_state_values
        
        # -----------------------------------------
        # 4. 결합 가중치 수정
        # -----------------------------------------
        # 4.1 신경망을 학습 모드로 전환
        self.model.train()

        # 4.2 손실함수를 계산 (smooth_l1_loss는 Huber 함수)
        # expected_state_action_values은
        # size가 [minibatch]이므로 unsqueeze하여 [minibatch*1]로 만든다
        loss = F.smooth_l1_loss(state_action_values,expected_state_action_values.unsqueeze(1))

        # 4.3 결합 가중치를 수정한다
        self.optimizer.zero_grad()  # 경사를 초기화
        loss.backward()  # 역전파 계산
        self.optimizer.step()  # 결합 가중치 수정
        
    def decide_action(self, state, episode):
        '''현재 상태에 따라 행동을 결정한다'''
        # ε-greedy 알고리즘에서 서서히 최적행동의 비중을 늘린다
        epsilon = 0.5 * (1 / (episode + 1))

        if epsilon <= np.random.uniform(0, 1):
            self.model.eval()  # 신경망을 추론 모드로 전환
            with torch.no_grad():
                action = self.model(state).max(1)[1].view(1, 1)
            # 신경망 출력의 최댓값에 대한 인덱스 = max(1)[1]
            # .view(1,1)은 [torch.LongTensor of size 1] 을 size 1*1로 변환하는 역할을 한다

        else:
            # 행동을 무작위로 반환(0 혹은 1)
            action = torch.LongTensor(
                [[random.randrange(self.num_actions)]])  # 행동을 무작위로 반환(0 혹은 1)
            # action은 [torch.LongTensor of size 1*1] 형태가 된다

        return action        
        

In [111]:
# CartPole 태스크의 에이전트 클래스. 봉 달린 수레 자체라고 보면 된다

class Agent:
    def __init__(self, num_states, num_actions):
        '''태스크의 상태 및 행동의 가짓수를 설정'''
        self.brain = Brain(num_states, num_actions)  # 에이전트의 행동을 결정할 두뇌 역할 객체를 생성

    def update_q_function(self):
        '''Q함수를 수정'''
        self.brain.replay()

    def get_action(self, state, episode):
        '''행동을 결정'''
        action = self.brain.decide_action(state, episode)
        return action

    def memorize(self, state, action, state_next, reward):
        '''memory 객체에 state, action, state_next, reward 내용을 저장'''
        self.brain.memory.push(state, action, state_next, reward)

In [112]:
# CartPole을 실행하는 환경 역할을 하는 클래스


class Environment:

    def __init__(self):
        self.env = gym.make(ENV)  # 태스크를 설정
        num_states = self.env.observation_space.shape[0]  # 태스크의 상태 변수 수(4)를 받아옴
        num_actions = self.env.action_space.n  # 태스크의 행동 가짓수(2)를 받아옴
        self.agent = Agent(num_states, num_actions)  # 에이전트 역할을 할 객체를 생성

        
    def run(self):
        '''실행'''
        episode_10_list = np.zeros(10)  # 최근 10에피소드 동안 버틴 단계 수를 저장함(평균 단계 수를 출력할 때 사용)
        complete_episodes = 0  # 현재까지 195단계를 버틴 에피소드 수
        episode_final = False  # 마지막 에피소드 여부
        frames = []  # 애니메이션을 만들기 위해 마지막 에피소드의 프레임을 저장할 배열

        for episode in range(10):  # 최대 에피소드 수만큼 반복
            observation = self.env.reset()  # 환경 초기화

            state = observation  # 관측을 변환없이 그대로 상태 s로 사용
            state = torch.from_numpy(state).type(torch.FloatTensor)  # NumPy 변수를 파이토치 텐서로 변환
            state = torch.unsqueeze(state, 0)  # size 4를 size 1*4로 변환

            for step in range(200):  # 1 에피소드에 해당하는 반복문

                if episode_final is True:  # 마지막 에피소드에서는 각 시각의 이미지를 frames에 저장한다
                    frames.append(self.env.render())

                action = self.agent.get_action(state, episode)  # 다음 행동을 결정

                # 행동 a_t를 실행하여 다음 상태 s_{t+1}과 done 플래그 값을 결정
                # action에 .item()을 호출하여 행동 내용을 구함
                # reward와 info는 사용하지 않으므로 _로 처리
                observation_next, _, done, _ = self.env.step(action.item())  
                
                # 보상을 부여하고 episode의 종료 판정 및 state_next를 설정한다
                if done:  # 단계 수가 200을 넘었거나 봉이 일정 각도 이상 기울면 done이 True가 됨
                    state_next = None  # 다음 상태가 없으므로 None으로

                    # 최근 10 에피소드에서 버틴 단계 수를 리스트에 저장
                    episode_10_list = np.hstack((episode_10_list[1:], step + 1))

                    if step < 195:
                        reward = torch.FloatTensor([-1.0])  # 도중에 봉이 쓰러졌다면 페널티로 보상 -1을 부여
                        complete_episodes = 0  # 연속 성공 에피소드 기록을 초기화
                    else:
                        reward = torch.FloatTensor([1.0])  # 봉이 서 있는 채로 에피소드를 마쳤다면 보상 1 부여
                        complete_episodes = complete_episodes + 1  # 연속 성공 에피소드 기록을 갱신
                else:
                    reward = torch.FloatTensor([0.0])  # 그 외의 경우는 보상 0을 부여
                    state_next = observation_next  # 관측 결과를 그대로 상태로 사용
                    state_next = torch.from_numpy(state_next).type(torch.FloatTensor)  # numpy 변수를 파이토치 텐서로 변환
                    state_next = torch.unsqueeze(state_next, 0)  # size 4를 size 1*4로 변환

                # 메모리에 경험을 저장
                self.agent.memorize(state, action, state_next, reward)

                # Experience Replay로 Q함수를 수정
                self.agent.update_q_function()

                # 관측 결과를 업데이트
                state = state_next

                # 에피소드 종료 처리
                if done:
                    print('%d Episode: Finished after %d steps：최근 10 에피소드의 평균 단계 수 = %.1lf' % (
                        episode, step + 1, episode_10_list.mean()))
                    break

            if episode_final is True:
                # 애니메이션 생성 및 저장
                display_frames_as_gif(frames)
                break

            # 10 에피소드 연속으로 195단계를 버티면 태스크 성공
            if complete_episodes >= 10:
                print('10 에피소드 연속 성공')
                episode_final = True  # 다음 에피소드에서 애니메이션을 생성

In [113]:
# 실행 엔트리 포인트
cartpole_env = Environment()
cartpole_env.run()

Sequential(
  (fc1): Linear(in_features=4, out_features=32, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=32, out_features=2, bias=True)
)
0 Episode: Finished after 10 steps：최근 10 에피소드의 평균 단계 수 = 1.0
1 Episode: Finished after 12 steps：최근 10 에피소드의 평균 단계 수 = 2.2
2 Episode: Finished after 8 steps：최근 10 에피소드의 평균 단계 수 = 3.0
action_batch =  tensor([[0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]])
self.model(state_batch) =  tensor([[ 0.1728,  0.0520],
        [ 0.1574,  0.0079],
        [ 0.1910,  0.0952],
        [ 0.1877,  0.0816],
        [ 0.1468, 

/tmp/ipykernel_804/3671214090.py:90: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755883846/work/aten/src/ATen/native/IndexingUtils.h:28.)
  next_state_values[non_final_mask] = self.model(non_final_next_states).max(1)[0].detach()



action_batch =  tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]])
self.model(state_batch) =  tensor([[ 0.1343,  0.0432],
        [ 0.1323,  0.0285],
        [ 0.1333,  0.0656],
        [ 0.1239, -0.0066],
        [ 0.1348,  0.0536],
        [ 0.1279,  0.0154],
        [ 0.1197, -0.0169],
        [ 0.1334,  0.0429],
        [ 0.1339,  0.0405],
        [ 0.1323,  0.0535],
        [ 0.1356,  0.0661],
        [ 0.1316,  0.0123],
        [ 0.1204, -0.0196],
        [ 0.1287,  0.0057],
        [ 0.1208, -0.0178],
        [ 0.1272,  0.0040],
        [ 0.1268,  0.0047],
        [ 0.1359,  0.0676],
        [ 0.1247, -0.0068],
       

self.model(state_batch) =  tensor([[ 0.1045,  0.0020],
        [ 0.1013,  0.0204],
        [ 0.0946,  0.0482],
        [ 0.0976,  0.0364],
        [ 0.0911,  0.0618],
        [ 0.1049,  0.0066],
        [ 0.0970,  0.0334],
        [ 0.0963,  0.0334],
        [ 0.1045,  0.0096],
        [ 0.0976,  0.0327],
        [ 0.1038,  0.0101],
        [ 0.0923,  0.0589],
        [ 0.1014,  0.0096],
        [ 0.0979,  0.0368],
        [ 0.1035,  0.0006],
        [ 0.1030,  0.0108],
        [ 0.1007,  0.0212],
        [ 0.1015,  0.0102],
        [ 0.1051, -0.0104],
        [ 0.1056, -0.0203],
        [ 0.0909,  0.0613],
        [ 0.1050,  0.0017],
        [ 0.1060, -0.0086],
        [ 0.1030,  0.0125],
        [ 0.0944,  0.0467],
        [ 0.0994,  0.0202],
        [ 0.1067, -0.0038],
        [ 0.1074, -0.0187],
        [ 0.1081, -0.0194],
        [ 0.1046, -0.0094],
        [ 0.0945,  0.0477],
        [ 0.0940,  0.0500]], grad_fn=<AddmmBackward0>)
action_batch =  tensor([[0],
        [0],
        

self.model(state_batch) =  tensor([[ 0.0807, -0.0080],
        [ 0.0631,  0.0210],
        [ 0.0769, -0.0006],
        [ 0.0570,  0.0314],
        [ 0.0402,  0.0526],
        [ 0.0624,  0.0227],
        [ 0.0918, -0.0254],
        [ 0.0914, -0.0275],
        [ 0.0824, -0.0088],
        [ 0.0877, -0.0198],
        [ 0.0488,  0.0451],
        [ 0.0476,  0.0411],
        [ 0.0896, -0.0252],
        [ 0.0394,  0.0518],
        [ 0.0373,  0.0526],
        [ 0.0798, -0.0085],
        [ 0.0913, -0.0238],
        [ 0.0548,  0.0342],
        [ 0.0940, -0.0267],
        [ 0.0823, -0.0127],
        [ 0.0696,  0.0083],
        [ 0.0688,  0.0083],
        [ 0.0463,  0.0428],
        [ 0.0468,  0.0445],
        [ 0.0894, -0.0245],
        [ 0.0914, -0.0261],
        [ 0.0777, -0.0011],
        [ 0.0695,  0.0090],
        [ 0.0824, -0.0114],
        [ 0.0909, -0.0242],
        [ 0.0866, -0.0175],
        [ 0.0385,  0.0550]], grad_fn=<AddmmBackward0>)
action_batch =  tensor([[0],
        [0],
        